In [1]:
import tensorflow as tf
import numpy as np

## 특이사항

- `tf.summary.scala` & `tf.summary.histogram` 이 어떻게 동작하는지.. `scala`는 안되다가 어쩌다가 되고, `histogram`은 안됨..
- 모델을 바꾸면 `tf.summary`에도 영향을 주는 듯함 

In [2]:
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [3]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



- `with tf.name_scope('layer1')` : 이렇게 묶인 블록은 한 계층 내부를 표현.
- `tf.Variable`의 name parameter : 텐서보드에서 해당 이름의 변수가 어디서 사용되는지 쉽게 확인할 수 있음

In [4]:
# 손실값을 추적하기 위한 코드
tf.summary.scalar('cost', cost)
tf.summary.histogram('Weights', W1)

<tf.Tensor 'Weights:0' shape=() dtype=string>

- `tf.summary.scalar` : 값이 하나인 텐서를 수집할 때 사용하는 코드

In [9]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./tb_model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    try : 
        saver.restore(sess, ckpt.model_checkpoint_path)
    except tf.errors.NotFoundError as e:
        print("remove a model directory because of NotFoundError!")
        !rm -rf ./tb_model
        sess.run(tf.global_variables_initializer())
else: 
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./tb_model/dnn.ckpt-200


책 87쪽 노트에 보면 model이 달라졌을 때, 모델을 불러오면 에러가 나는 것을 알 수 있다. 그래서 나는 에러가 났을 경우에 모델 디렉토리를 삭제하는 매직 커맨드 코드를 추가했다.

In [10]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

- `tf.summary.merge_all` : 앞서 지정한 텐서들을 모두 수집
- `tf.summary.FileWriter` : 그래프와 텐서의 값을 저장할 디렉토리 설정

In [11]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if step % 10 == 0:
        print('Step : %d' % sess.run(global_step), 
             'Cost : %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step : 201 Cost : 0.549
Step : 211 Cost : 0.549
Step : 221 Cost : 0.549
Step : 231 Cost : 0.549
Step : 241 Cost : 0.549
Step : 251 Cost : 0.549
Step : 261 Cost : 0.549
Step : 271 Cost : 0.549
Step : 281 Cost : 0.549
Step : 291 Cost : 0.549


- `writer.add_summary` : 텐서들의 값을 저장하는 코드. 특정 시점에 (여기서는 매단계에서) 값들을 수집하고 저장한다.

In [8]:
saver.save(sess, './tb_model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('prediction: ', sess.run(prediction, feed_dict={X: x_data}))
print('target: ', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("accuracy : %.2f" % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

prediction:  [0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2]
target:  [0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2 0 1 2 0 0 2]
accuracy : 100.00
